In [ ]:
%pip install geopandas plotly shapely seaborn

In [ ]:
from datetime import date
import numpy as np
import seaborn as sns
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
from shapely.geometry import Point
from matplotlib.colors import ListedColormap, BoundaryNorm

In [ ]:
from environment import dh, pio_renderer
if pio_renderer is not None:
    pio.renderers.default = pio_renderer

In [ ]:
PROJECT_NAME = "AreaVerde"

# **Functions**

In [ ]:
def Heatmap(data, annot, colors, bounds):
  plt.figure(figsize=(14, 10))
  # colors = ['#d1351d', '#006d2c']
  cmap = ListedColormap(colors)
  # max_value = 680971
  # bounds = [0,1, max_value]
  norm = BoundaryNorm(bounds, cmap.N)

  sns.heatmap(
      data,
      cmap=cmap,
      norm=norm,
      linewidths=0.6,
      annot=annot,
      fmt='d',
      cbar=False,
  )

  plt.tight_layout()
  plt.show()

In [ ]:
def create_map(gates, color_map, map_style="open-street-map", zoom=12, width=1200, height=1000):
    filtered_data = gdf[gdf['gate'].isin(gates)]
    filtered_data['lon'] = filtered_data.geometry.x
    filtered_data['lat'] = filtered_data.geometry.y

    fig = px.scatter_mapbox(
        filtered_data,
        lat="lat",
        lon="lon",
        hover_name="gate",
        hover_data={"vehicle": True, "Percentuale": True, "count": True, "lat": False, "lon": False},
        color="gate",
        color_discrete_map=color_map,
        mapbox_style=map_style,
        size="Percentuale",
        size_max=15,
        zoom=zoom,
        # width=width,
        height=height,
        text="gate"
    )

    return fig

In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [ ]:
def create_plt(ax, gate, index, color, zero_counts_by_hour_gate):
    ax.plot(zero_counts_by_hour_gate.index, zero_counts_by_hour_gate[gate],
            marker='o', linestyle='-', label=f'gate {gate}', color=color)
    ax.set_ylabel('Number of Zero')
    ax.set_xticks(range(0, 24))
    ax.legend(title='gate', bbox_to_anchor=(1.05, 1), loc='upper left')

# Load data

In [ ]:
project = dh.get_or_create_project(PROJECT_NAME)

gates = project.get_dataitem("gates").as_df()
data = project.get_dataitem("gate_data").as_df()
# Join coordinates
gate_data = pd.merge(data, gates, on="gate", how="left", validate="many_to_one").drop(columns=["ID", "Indirizzo", "Settore", "Link google maps"])
# reconvert string to datetime
gate_data["Data"] = pd.to_datetime(gate_data["Data"])

# **Seasonal**

In [ ]:
gate_data['Season'] = gate_data['Data'].dt.month.apply(get_season)

In [ ]:
total_vahicle_s = gate_data[["Season", "gate", "count"]].groupby(["Season", "gate"], as_index=False).sum()
total_vahicle_s = total_vahicle_s.pivot_table(index="Season", columns="gate", values="count", fill_value=0).astype(int)
total_vahicle_s

In [ ]:
total_vahicle_s = total_vahicle_s.T

In [ ]:
Heatmap(total_vahicle_s, annot=True, colors=['#d1351d', '#006d2c'], bounds=[0,1, 680971])

# **Monthly**

In [ ]:
gate_data['date'] = gate_data['Data'].dt.date

In [ ]:
gate_data['Month'] = gate_data['Data'].dt.to_period('M')
total_vahicle_m = gate_data[["Month", "gate", "count"]].groupby(["Month", "gate"], as_index=False).sum()
total_vahicle_m = total_vahicle_m.pivot_table(index="Month", columns="gate", values="count", fill_value=0).astype(int)
total_vahicle_m

In [ ]:
total_vahicle_m = total_vahicle_m.T

In [ ]:
Heatmap(total_vahicle_m, annot=True, colors=['#d1351d', '#006d2c'], bounds=[0,1, 680971])

# **Daily**

In [ ]:
gate_data['date'] = gate_data['Data'].dt.date

In [ ]:
daily_total_vahicle = gate_data[["date", "gate", "count"]].groupby(["date", "gate"], as_index=False).sum()
daily_total_vahicle = daily_total_vahicle.pivot_table(index="date", columns="gate", values="count", fill_value=0).astype(int)
daily_total_vahicle

In [ ]:
Heatmap(daily_total_vahicle,annot=False,colors=['#d1351d', '#006d2c'], bounds=[0,1, 680971])

# **Hourly**

In [ ]:
hourly_total_vahicle = gate_data[["Data", "gate", "count"]].groupby(["Data", "gate"], as_index=False).sum()
hourly_total_vahicle = hourly_total_vahicle.pivot_table(index="Data", columns="gate", values="count", fill_value=0).astype(int)

In [ ]:
hourly_total_vahicle

In [ ]:
daily_summary = (hourly_total_vahicle > 0).groupby(hourly_total_vahicle.index.date).sum()
daily_status = pd.DataFrame(index=daily_summary.index, columns=daily_summary.columns)

daily_status[(daily_summary >= 24)] = 'green'  # All 24 hours have non-zero data
daily_status[(daily_summary >= 18) & (daily_summary < 24)] = 'yellow'  # Some hours have zero data
daily_status[(daily_summary > 0) & (daily_summary < 18)] = 'orange'  # Some hours have zero data
daily_status[(daily_summary == 0)] = 'red'  # All hours are zero
color_mapping = {'red': 0, 'orange': 1, 'yellow': 2, 'green': 3}
daily_status = daily_status.replace(color_mapping)

In [ ]:
daily_status

In [ ]:
Heatmap(daily_status,annot=False,colors = ['red', 'orange', 'yellow', 'green'],bounds=[-0.5, 0.5, 1.5, 2.5, 3.5] )

# **Hourly Passing Trend**

In [ ]:
map_data = gate_data.copy()

subset_vehicles = ['Autoveicolo','Autocarro','Motoveicolo']
map_data = map_data[map_data['vehicle'].isin(subset_vehicles)]
map_data['Percentuale'] = map_data.groupby(['Data', 'vehicle'])['count'].transform(lambda x: np.round(x / x.sum(), 2))

geometry = [Point(xy) for xy in zip(map_data['Longitudine'], map_data['Latitudine'])]
gdf = gpd.GeoDataFrame(map_data, geometry=geometry)

In [ ]:
zero_counts_by_hour_gate = hourly_total_vahicle.eq(0).groupby(hourly_total_vahicle.index.hour).sum()
zero_counts_by_hour_gate

In [ ]:
gates = zero_counts_by_hour_gate.columns.tolist()
color_groups = {
    "blue": ['Massarenti', 'Dozza', 'Don Sturzo', 'Triumvirato'],
    "orange": ['Monte Donato', 'Boninsegna', 'Stalingrado', 'Guelfa', 'San Luca', 'Casaglia'],
    "green": ['Alberto Mario', 'Arcoveggio', 'Zanardi', 'Roma', 'Castiglione', 'Terrapieno', 'Porrettana/Bonvicini', 'Lenin', 'Due Madonne', 'Saragat'],
    "red": ['Della Pietra', 'Dozza', 'Triumvirato', 'Don Sturzo', 'Togliatti', 'S.Donato'],
    "purple": ['Corelli_2 Sud', 'Corelli_1 Nord', 'Scandellara'],
    "brown": ['Asse Attrezzato', 'Sanzio']
}
color_map = {gate: color for color, gates in color_groups.items() for gate in gates}

fig = create_map(gates,color_map)
fig.show()


In [ ]:
gates_sets = [
    ['Massarenti', 'Dozza', 'Don Sturzo', 'Triumvirato'],
    ['Monte Donato', 'Boninsegna', 'Stalingrado', 'Guelfa', 'San Luca', 'Casaglia'],
    ['Alberto Mario', 'Arcoveggio', 'Zanardi', 'Roma', 'Castiglione', 'Terrapieno',
     'Porrettana/Bonvicini', "Lenin", 'Due Madonne', "Saragat", "Castiglione", "Arcoveggio"],
    ['Dozza', 'Triumvirato', 'Della Pietra', 'Don Sturzo', 'Togliatti', 'S.Donato'],
    ['Corelli_2 Sud', 'Corelli_1 Nord', "Scandellara"],
    ['Asse Attrezzato', 'Sanzio']
]

colors = ["blue", "red", "green", "orange", "purple", 'brown']

fig, axes = plt.subplots(3, 2, figsize=(18, 15))
axes = axes.flatten()

for (i, gates), color in zip(enumerate(gates_sets), colors):
    ax = axes[i]
    for j, gate in enumerate(gates):
        create_plt(ax, gate, j, color, zero_counts_by_hour_gate)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14, 8))

for gate in zero_counts_by_hour_gate.columns:
    plt.plot(zero_counts_by_hour_gate.index, zero_counts_by_hour_gate[gate], marker='o', linestyle='-', label=f'gate {gate}')

plt.ylabel('Number of Zero')
plt.xticks(range(0, 24))

plt.legend(title='gate', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.grid()
plt.show()

# **June In Depth Analysis**

In [ ]:
sel_date_min = date(2024,  6,  1)
sel_date_max = date(2024, 6,  30)
sel_gates = ['Alberto Mario', 'Arcoveggio', 'Corelli_1 Nord', 'Don Sturzo',
             'Dozza', 'Massarenti', 'Saragat', 'Triumvirato', 'Zanardi']

In [ ]:
daily_status_sel = daily_status.copy()
daily_status_sel = daily_status_sel[sel_date_min:sel_date_max]
daily_status_sel

In [ ]:
Heatmap(daily_status_sel,annot=False,colors=['red', 'orange', 'yellow', 'green'], bounds=[-0.5, 0.5, 1.5, 2.5, 3.5])

In [ ]:
total_vahicle_h = hourly_total_vahicle.copy().reset_index()
total_vahicle_h['hour'] = total_vahicle_h['Data'].dt.hour
total_vahicle_h = total_vahicle_h.drop(columns=['Data'])
total_vahicle_h = total_vahicle_h.groupby(['hour'], as_index=False).agg(list)#agg(lambda x: list(x / max(x)))

total_vahicle_h_sel = hourly_total_vahicle[sel_date_min:sel_date_max][sel_gates].copy().reset_index()
total_vahicle_h_sel['hour'] = total_vahicle_h_sel['Data'].dt.hour
total_vahicle_h_sel = total_vahicle_h_sel.drop(columns=['Data'])
total_vahicle_h_sel = total_vahicle_h_sel.groupby(['hour'], as_index=False).agg(list)#agg(lambda x: list(x / max(x)))

total_vahicle_h_sel

In [ ]:
#gate = "Triumvirato"
#gate = "Arcoveggio"
#gate = "Terrapieno"
gate = "Alberto Mario"
#gate = "Boninsegna"
fig, axs = plt.subplots(6, 4, layout='constrained', figsize=(20, 20))
for i in range(0,24):
    ax = axs[i//4][i%4]
    ax.hist(total_vahicle_h[total_vahicle_h['hour'] == i ][gate], histtype="step", bins=2000, cumulative=True, density=True)
    if gate in total_vahicle_h_sel.columns:
        ax.hist(total_vahicle_h_sel[total_vahicle_h_sel['hour'] == i ][gate], histtype="step", bins=2000, cumulative=True, density=True)

In [ ]:
#ora = 4
hour = 8
fig, axs = plt.subplots(7, 4, layout='constrained', figsize=(20, 20))
for i,c in enumerate(hourly_total_vahicle.columns[1:]):
    ax = axs[i//4][i%4]
    ax.hist(total_vahicle_h[total_vahicle_h['hour'] == hour ][c], histtype="step", bins=2000, cumulative=True, density=True)
    if c in total_vahicle_h_sel.columns:
        ax.hist(total_vahicle_h_sel[total_vahicle_h_sel['hour'] == hour ][c], histtype="step", bins=2000, cumulative=True, density=True)


In [ ]:
start = date(2024, 6,1)
end = date(2024, 6, 30)
total_vahicle_h = hourly_total_vahicle[start:end][sel_gates]

In [ ]:
total_vahicle_h

In [ ]:
total_vahicle_h.plot(figsize=(18, 10))
# Saturday Sunday

In [ ]:
start = date(2024, 6,7)
end = date(2024, 6, 10)
sel_total_vahicle = hourly_total_vahicle[start:end][sel_gates]
sel_total_vahicle.plot(figsize=(16,10))

In [ ]:
total_vahicle_h = total_vahicle_h.reset_index()

total_vahicle_h['Weekday'] = total_vahicle_h['Data'].dt.day_name()
total_vahicle_h

In [ ]:
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
grouped_data = total_vahicle_h.groupby('Weekday').mean().reindex(weekday_order)

In [ ]:
grouped_data.drop(columns=['Data']).plot(figsize=(14, 10))